In [1]:
!pip install fer

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
!pip install torch torchvision

In [1]:
!pip install opencv-python opencv-python-headless

In [3]:
!pip install torch torchvision

In [4]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 13.2 MB/s eta 0:00:00


In [5]:
!pip install filterpy scikit-learn lap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110459 sha256=f7c46634fc7333b382884fd2e62c7c4afe256a22681e426793bac2c38205544c
  Stored in directory: /root/.cache/pip/wheels/0f/0c/ea/218f266af4ad626897562199fbbcba521b8497303200186102
  Created wheel for lap: filename=lap-0.4.0-cp310-cp310-linux_x86_64.whl size=1628943 sha256=d0473ada919e401049b686c300832f16a6983f4494858712ac6b12b93b481962
  Stored in directory: /root/.cache/pip/wheels/00/42/2e/9dfe19270eea279d79e84767ff0d7b8082c3bf776cad00e83d
Successfully built filterpy lap


In [6]:
!pip install matplotlib

In [7]:
import torch
import cv2
import ultralytics

print(f"PyTorch Version: {torch.__version__}")
print(f"OpenCV Version: {cv2.__version__}")
print(f"Ultralytics Version: {ultralytics.__version__}")

PyTorch Version: 2.2.2+cu121
OpenCV Version: 4.10.0
Ultralytics Version: 8.2.85


In [8]:
from ultralytics import YOLO
import cv2
from google.colab import files
from fer import FER
import numpy as np
import io

# Initialize the YOLO model and Emotion Detector
model = YOLO('yolov8n.pt')
emotion_detector = FER(mtcnn=True)  # Using MTCNN for better face detection

# Upload video file directly
uploaded = files.upload()

# Check if any file is uploaded
if not uploaded:
    print("Error: No file uploaded.")
    exit()

# Get the uploaded file's name
input_filename = list(uploaded.keys())[0]
input_file = uploaded[input_filename]

# Load the video from the uploaded file
input_video_path = '/content/' + input_filename
with open(input_video_path, 'wb') as f:
    f.write(input_file)

cap = cv2.VideoCapture(input_video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Video writer setup
output_video_path = '/content/output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (frame_width, frame_height))

# Class ID for 'person' (YOLOv8) and child/adult threshold
PERSON_CLASS_ID = 0
CHILD_THRESHOLD = 280

# Initialize tracking variables
next_id = 0
trackers = {}
id_to_label = {}

# Function to calculate IoU (Intersection over Union)
def iou(box1, box2):
    x1_t, y1_t, x2_t, y2_t = box1
    x1, y1, x2, y2 = box2

    # Calculate intersection
    xi1, yi1 = max(x1, x1_t), max(y1, y1_t)
    xi2, yi2 = min(x2, x2_t), min(y2, y2_t)
    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)

    # Calculate union
    box1_area = (x2_t - x1_t) * (y2_t - y1_t)
    box2_area = (x2 - x1) * (y2 - y1)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    person_boxes = []

    for result in results:
        boxes = result.boxes
        classes = boxes.cls
        for box, cls in zip(boxes, classes):
            if int(cls) == PERSON_CLASS_ID:
                person_boxes.append(box)

    current_frame_ids = []
    for box in person_boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
        height = y2 - y1
        label = 'Child' if height < CHILD_THRESHOLD else 'Therapist'
        color = (0, 255, 255) if label == 'Child' else (0, 255, 0)

        matched_id = None
        for tracker_id, prev_box in trackers.items():
            if iou(prev_box, (x1, y1, x2, y2)) > 0.3:
                matched_id = tracker_id
                break

        if matched_id is None:
            matched_id = next_id
            next_id += 1

        trackers[matched_id] = (x1, y1, x2, y2)
        id_to_label[matched_id] = label
        current_frame_ids.append(matched_id)

        # Draw bounding box and ID label
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"ID {matched_id}: {label}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Detect emotions within the person ROI
        person_roi = frame[y1:y2, x1:x2]
        if person_roi.size != 0:
            try:
                emotions = emotion_detector.detect_emotions(person_roi)
                if emotions:
                    top_emotion, score = emotion_detector.top_emotion(person_roi)
                    if top_emotion:
                        emotion_label = f"{top_emotion} ({int(score * 100)}%)"
                        cv2.putText(frame, emotion_label, (x1, y2 + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            except Exception as e:
                print(f"Error detecting emotions: {e}")

    # Remove trackers that are not in the current frame
    trackers = {tracker_id: box for tracker_id, box in trackers.items() if tracker_id in current_frame_ids}

    # Write the processed frame to the output video
    out.write(frame)

# Release everything
cap.release()
out.release()
cv2.destroyAllWindows()

print("\nVideo processing complete. The output is saved as 'output_video.avi'.")

# Download the video
files.download(output_video_path)


100%|██████████| 6.25M/6.25M [00:00<00:00, 51.8MB/s]


Streaming output truncated to the last 5000 lines.

0: 384x640 2 persons, 138.1ms
Speed: 2.3ms preprocess, 138.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 134.9ms
Speed: 2.8ms preprocess, 134.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 136.7ms
Speed: 2.2ms preprocess, 136.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 196.9ms
Speed: 2.4ms preprocess, 196.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 205.4ms
Speed: 4.2ms preprocess, 205.4ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 134.7ms
Speed: 2.2ms preprocess, 134.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 137.7ms
Speed: 2.3ms preprocess, 137.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 139.8ms
Speed: 3.3ms pr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>